### Experiential Learning 
#### HR Interview Prep Bot

In [39]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher

syn_df = pd.read_csv("synonym.csv")
print(syn_df.columns)


Index(['question', 'synonym_phrases'], dtype='object')


In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher

# ---------- LOAD MAIN DATASET ----------
df = pd.read_csv("hr_dataset.csv")  # Must contain: question, expected_answer, example_answer

# Clean text helper
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z ]', '', text)
    return text.strip()

# Add cleaned question column
df['clean_question'] = df['question'].apply(clean_text)
questions = df['clean_question'].tolist()

# ---------- LOAD SYNONYM CSV ----------
syn_df = pd.read_csv("synonym.csv")  # Must contain: question, synonym_phrases

synonym_map = {}

for idx, row in syn_df.iterrows():
    main_q = clean_text(row['question'])
    synonyms = str(row['synonym_phrases']).split(",")

    for s in synonyms:
        s_clean = clean_text(s)
        if s_clean != "":
            synonym_map[s_clean] = main_q

# ---------- TF-IDF VECTOR ----------
vectorizer = TfidfVectorizer()
question_vectors = vectorizer.fit_transform(questions)

# ---------- MATCH FUNCTION ----------
def get_best_match(user_input):
    user_input_clean = clean_text(user_input)

    # --- SYNONYM LOOKUP FIRST ---
    if user_input_clean in synonym_map:
        mapped_q_clean = synonym_map[user_input_clean]
        match_row = df[df['clean_question'] == mapped_q_clean]

        if not match_row.empty:
            row = match_row.iloc[0]
            return row['question'], row['expected_answer'], row['example_answer'], 1.0

    # --- TF-IDF SIMILARITY ---
    user_vec = vectorizer.transform([user_input_clean])
    cosine_scores = cosine_similarity(user_vec, question_vectors).flatten()

    # --- FUZZY SIMILARITY ---
    fuzzy_scores = [SequenceMatcher(None, user_input_clean, q).ratio() for q in questions]

    # --- COMBINED SCORE ---
    combined_scores = (0.65 * cosine_scores) + (0.35 * pd.Series(fuzzy_scores))
    best_index = combined_scores.idxmax()
    best_score = combined_scores[best_index]

    if best_score < 0.10:
        return None, None, None, None

    row = df.iloc[best_index]
    return row['question'], row['expected_answer'], row['example_answer'], best_score

# ---------- CHAT LOOP ----------
print("\n🤖 HR Interview Prep Bot is Ready! Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Bot: Thank you! All the best for your interview 😊")
        break

    matched_q, expected, example, score = get_best_match(user_input)

    if matched_q is None:
        print("\nBot: ❗ I didn't understand that. Try another word.\n")
    else:
        print(f"\n🔹 Matched Question: {matched_q}")
        print(f"\n🔹 Expected Answer:\n {expected}")
        print(f"\n✅ Example Answer:\n {example}\n")



🤖 HR Interview Prep Bot is Ready! Type 'exit' to stop.



You:  intro



Bot: ❗ I didn't understand that. Try another word.



You:  introduction



Bot: ❗ I didn't understand that. Try another word.



...


🔹 Matched Question: Tell me about yourself.

🔹 Expected Answer:
 Start with your name, education, skills, and career interests.

✅ Example Answer:
 My name is Sakshi. I am currently pursuing my degree in Computer Science. I enjoy learning new things and I have good communication and teamwork skills.



You:  weekness



🔹 Matched Question: What are your weaknesses?

🔹 Expected Answer:
 Share a small weakness and show how you are improving it.

✅ Example Answer:
 Sometimes I get nervous while speaking in front of many people, but I am improving this by practicing speaking in group discussions.



You:  hobbi



Bot: ❗ I didn't understand that. Try another word.

